The city of New York does restaurant inspections and assigns a grade.

In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
!aws s3 sync s3://dataincubator-course/coursedata/ . --exclude '*' --include 'nyc_inspection.db'

download: s3://dataincubator-course/coursedata/nyc_inspection.db to ./nyc_inspection.db


In [5]:
#load the pre-existing tables
%load_ext sql
%sql sqlite:///nyc_inspection.db

'Connected: @nyc_inspection.db'

In [3]:
%%sql
SELECT * FROM sqlite_master WHERE "type"='table';

 * sqlite:///nyc_inspection.db
Done.


type,name,tbl_name,rootpage,sql
table,writer,writer,2,"CREATE TABLE writer (first_name, last_name, year)"
table,webextract,webextract,3,"CREATE TABLE webextract ( ""index"" BIGINT, ""CAMIS"" BIGINT, ""DBA"" TEXT, ""BORO"" BIGINT, ""BUILDING"" TEXT, ""STREET"" TEXT, ""ZIPCODE"" FLOAT, ""PHONE"" TEXT, ""CUISINECODE"" BIGINT, ""INSPDATE"" TEXT, ""ACTION"" TEXT, ""VIOLCODE"" TEXT, ""SCORE"" FLOAT, ""CURRENTGRADE"" TEXT, ""GRADEDATE"" TEXT, ""RECORDDATE"" TEXT)"
table,violations,violations,24393,"CREATE TABLE violations ( ""index"" BIGINT, ""STARTDATE"" TEXT, ""ENDDATE"" TEXT, ""CRITICALFLAG"" TEXT, ""VIOLATIONCODE"" TEXT, ""VIOLATIONDESC"" TEXT)"
table,cuisine,cuisine,24426,"CREATE TABLE cuisine ( ""index"" BIGINT, ""CUISINECODE"" BIGINT, ""CODEDESC"" TEXT)"
table,boroughs,boroughs,24428,"CREATE TABLE boroughs ( id INT, name TEXT)"


In [57]:
%%sql
PRAGMA table_info(webextract)

 * sqlite:///nyc_inspection.db
Done.


cid,name,type,notnull,dflt_value,pk
0,index,BIGINT,0,None,0
1,CAMIS,BIGINT,0,None,0
2,DBA,TEXT,0,None,0
3,BORO,BIGINT,0,None,0
4,BUILDING,TEXT,0,None,0
5,STREET,TEXT,0,None,0
6,ZIPCODE,FLOAT,0,None,0
7,PHONE,TEXT,0,None,0
8,CUISINECODE,BIGINT,0,None,0
9,INSPDATE,TEXT,0,None,0


In [262]:
%%sql

SELECT * FROM webextract WHERE SCORE IS NULL  LIMIT 50

 * sqlite:///nyc_inspection.db
(sqlite3.OperationalError) no such column: SCORE [SQL: 'SELECT * FROM boroughs WHERE SCORE IS NULL  LIMIT 50'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [5]:
!sqlite3 cmd """\
DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"""

William|Shakespeare|1616
Francis|Fitzgerald|1896


In [6]:
%%sql 
DROP TABLE IF EXISTS writer;
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);

SELECT * FROM writer;

 * sqlite:///nyc_inspection.db
Done.
Done.
1 rows affected.
1 rows affected.
Done.


first_name,last_name,year
William,Shakespeare,1616
Francis,Fitzgerald,1896


In [ ]:
result

Inspections (`CAMIS`, `INSPDATE` pairs) that do not have a score - i.e. where none of the rows with those (`CAMIS`, `INSPDATE`) values has a score. The corresponding rows were remove from the data set for the rest of the project.

In [21]:
%%sql
SELECT IFNULL((SELECT SCORE FROM webextract LIMIT 1), 1)

 * sqlite:///nyc_inspection.db
Done.


"IFNULL((SELECT SCORE FROM webextract LIMIT 1), 1)"
2.0


In [23]:
%%sql

SELECT CAMIS, INSPDATE,SCORE FROM webextract WHERE SCORE IS NULL AND NOT INSPDATE IS NULL LIMIT 5

 * sqlite:///nyc_inspection.db
Done.


CAMIS,INSPDATE,SCORE
30075445,2011-04-27 00:00:00,None
30075445,2011-11-12 00:00:00,None
30075445,2011-11-12 00:00:00,None
40356483,2013-08-03 00:00:00,None
30075445,2012-05-03 00:00:00,None


In [177]:
%%sql
SELECT COUNT(*)
FROM ( 
SELECT CAMIS,INSPDATE, SUM(SCORE) as mysum
FROM webextract
GROUP BY CAMIS,INSPDATE
) as web
WHERE web.mysum IS NULL
LIMIT 5


 * sqlite:///nyc_inspection.db
Done.


COUNT(*)
8255


In [6]:
%%sql
DROP TABLE IF EXISTS webextract2;
CREATE TABLE webextract2 AS
    SELECT
        web.*
    FROM
        webextract as web,
        (SELECT *
         FROM (
             SELECT CAMIS as cam,INSPDATE as ins, SUM(SCORE) as mysum
             FROM webextract
             GROUP BY CAMIS,INSPDATE
         ) as pair
         WHERE NOT pair.mysum IS NULL) as cond
    WHERE
        web.CAMIS == cond.cam AND
        web.INSPDATE==cond.ins


 * sqlite:///nyc_inspection.db
Done.
Done.


[]

List of tuples of the form:

    (zip code, mean score, number of restaurants)

for each of the 87 zip codes in the city with over 100 restaurants using the score from the latest inspection date for each restaurant. Sorted in ascending order by mean score.


In [10]:
%%sql
SELECT COUNT(*)
FROM(
SELECT COUNT(*)
FROM webextract2
GROUP BY CAMIS
)

 * sqlite:///nyc_inspection.db
Done.


COUNT(*)
25232


In [226]:
%%sql
SELECT COUNT(*)
FROM(
    SELECT myquery.zip,COUNT(myquery.count_dba) as final_count,AVG(myquery.mean_score)
    FROM(
        SELECT ZIPCODE as zip, COUNT(DBA) as count_dba,SUM(SCORE)/COUNT(SCORE) as mean_score
        FROM webextract2
        GROUP BY CAMIS
    ) as myquery
    GROUP BY
    myquery.zip
) myquery2
WHERE
    myquery2.final_count>100


 * sqlite:///nyc_inspection.db
Done.


COUNT(*)
87


In [209]:
%%sql
SELECT *
FROM(
    SELECT myquery.zip, COUNT(myquery.count_dba) as final_count, AVG(myquery.mean_score), insp_date
    FROM(
        SELECT ZIPCODE as zip, COUNT(DBA) as count_dba,AVG(SCORE) as mean_score, INSPDATE as insp_date
        FROM webextract2
        GROUP BY CAMIS
    ) as myquery
    GROUP BY
        myquery.zip
) as myquery2
WHERE
    myquery2.final_count>100

ORDER BY
    myquery2.insp_date
    

 * sqlite:///nyc_inspection.db
Done.


zip,final_count,AVG(myquery.mean_score),insp_date
11430.0,145,14.866404530245513,2014-04-14 00:00:00
11361.0,112,16.453863815548353,2014-05-27 00:00:00
10024.0,195,18.449213472074625,2014-06-09 00:00:00
11204.0,120,18.20725776083231,2014-06-12 00:00:00
10023.0,192,17.529437260412855,2014-06-20 00:00:00
11231.0,152,17.26525825077795,2014-07-02 00:00:00
11375.0,190,17.8703618802192,2014-07-08 00:00:00
11206.0,141,17.637406910273423,2014-07-10 00:00:00
10021.0,149,19.47690903069643,2014-07-11 00:00:00
10463.0,126,17.06368562767339,2014-07-18 00:00:00


In [219]:
%%sql
SELECT ZIPCODE as zip, CAMIS, SCORE, MAX(INSPDATE) as insp_date
FROM webextract2
GROUP BY CAMIS
LIMIT 5

 * sqlite:///nyc_inspection.db
Done.


zip,CAMIS,SCORE,insp_date
10462.0,30075445,2.0,2014-03-03 00:00:00
11225.0,30112340,23.0,2014-07-01 00:00:00
10019.0,30191841,11.0,2013-07-22 00:00:00
11224.0,40356018,5.0,2014-06-10 00:00:00
11374.0,40356068,12.0,2014-01-29 00:00:00


In [228]:
%%sql
SELECT myquery2.zip, myquery2.mean_score,myquery2.final_count
FROM(
    SELECT myquery.zip, COUNT(myquery.res_id) as final_count, AVG(myquery.last_score) as mean_score, insp_date
    FROM(
        SELECT ZIPCODE as zip, CAMIS as res_id, SCORE as last_score, MAX(INSPDATE) as insp_date
        FROM webextract2
        GROUP BY CAMIS
    ) as myquery
    GROUP BY
        myquery.zip
) as myquery2
WHERE
    myquery2.final_count>100

ORDER BY
    myquery2.mean_score

 * sqlite:///nyc_inspection.db
Done.


myquery2.zip,myquery2.mean_score,myquery2.final_count
10451.0,9.142857142857142,154
10001.0,9.14949494949495,495
11430.0,9.5,145
11236.0,9.830188679245284,106
10472.0,10.130841121495328,107
11234.0,10.186206896551724,145
11217.0,10.345528455284553,246
11368.0,10.480286738351255,280
10306.0,10.533333333333333,105
11237.0,10.53448275862069,175


In [260]:
def score_by_zipcode():
    return c

grader.score('sql__score_by_zipcode', score_by_zipcode)

Your score:  0.999999999999999


## Question 3: Mapping scores


The above are not terribly enlightening.  Use [CartoDB](http://cartodb.com/) to produce a map of average scores by zip code.  You can sign up for a free trial.

You will have to use their wizard to plot the data by [zip code](https://carto.com/learn/guides/analysis/georeference). You will need to specify "USA" in the country field.  Then use the "share" button to return a link of the form [https://x.cartodb.com/](https://x.cartodb.com/).

**For fun:** How do JFK, Brighton Beach, Liberty Island (home of the Statue of Liberty), Financial District, Chinatown, and Coney Island fare?

**For more fun:** Plot restaurants as pins on the map, allowing the user to filter by "low", "middling", or "high"-scoring restaurants. You can use a CASE WHEN statement to create the different groups based on score thresholds.

In [261]:
def score_by_map():
    # must be url of the form https://x.cartodb.com/...
    return "https://smonjezi.carto.com/builder/df6a9df7-7f86-450b-af29-e83523513d51/embed"

grader.score('sql__score_by_map', score_by_map)

Your score:  1.0


## Question 4: Score by borough

Return a list of tuples of the form:

    (borough, mean score, number of restaurants)

for each of the city's five boroughs. Use the latest score for each restaurant. Sort the list in ascending order by grade.

**Hint:** You will have to perform a join with the `boroughs` table. The borough names should be reported in ALL CAPS.

**Checkpoint:**
- Total restaurants in valid boroughs: 24,350

In [28]:
%%sql
SELECT COUNT(*)
FROM (
SELECT *
FROM webextract2
WHERE BORO >0
GROUP BY CAMIS
)


 * sqlite:///nyc_inspection.db
Done.


COUNT(*)
24350


In [30]:
%%sql
DROP TABLE IF EXISTS webextract3;
CREATE TABLE webextract3 AS
    SELECT
        web.*
    FROM
        webextract2 as web,
        (SELECT CAMIS as cam, BORO as brgh
         FROM webextract2
         WHERE BORO >0
         GROUP BY CAMIS) as cond
    WHERE
        web.CAMIS == cond.cam AND
        web.BORO==cond.brgh


 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [33]:
%%sql
SELECT COUNT(*) FROM webextract3

 * sqlite:///nyc_inspection.db
Done.


COUNT(*)
522710


In [34]:
%%sql
SELECT COUNT(*)
FROM (
SELECT *
FROM webextract3
GROUP BY CAMIS
)

 * sqlite:///nyc_inspection.db
Done.


COUNT(*)
24350


In [61]:
%%sql

SELECT * FROM webextract WHERE SCORE IS NULL LIMIT 50

 * sqlite:///nyc_inspection.db
Done.


index,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
56171,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE,10462.0,7188924968,8,2011-04-27 00:00:00,F,16A,None,None,None,2014-09-04 06:01:28.403000000
56193,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE,10462.0,7188924968,8,2011-11-12 00:00:00,F,16A,None,None,None,2014-09-04 06:01:28.403000000
56194,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE,10462.0,7188924968,8,2011-11-12 00:00:00,F,16B,None,None,None,2014-09-04 06:01:28.403000000
56200,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234.0,7184443838,27,2013-08-03 00:00:00,F,16A,None,None,None,2014-09-04 06:01:28.403000000
56204,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE,10462.0,7188924968,8,2012-05-03 00:00:00,B,None,None,None,None,2014-09-04 06:01:28.403000000
56208,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE,10462.0,7188924968,8,2013-06-01 00:00:00,F,16B,None,None,None,2014-09-04 06:01:28.403000000
56226,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE,10462.0,7188924968,8,2011-12-21 00:00:00,F,16B,None,None,None,2014-09-04 06:01:28.403000000
56227,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE,10462.0,7188924968,8,2012-12-31 00:00:00,F,16B,None,None,None,2014-09-04 06:01:28.403000000
56240,40356151,BRUNOS ON THE BOULEVARD,4,88-25,ASTORIA BOULEVARD,11369.0,7183350505,3,2012-02-10 00:00:00,E,16B,None,None,None,2014-09-04 06:01:28.403000000
56253,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019.0,2122452912,3,2010-09-23 00:00:00,F,15L,None,None,None,2014-09-04 06:01:28.403000000


In [7]:
%%sql

SELECT * FROM boroughs

 * sqlite:///nyc_inspection.db
Done.


id,name
1,MANHATTAN
2,THE BRONX
3,BROOKLYN
4,QUEENS
5,STATEN ISLAND


In [42]:
%%sql
--work here
SELECT myquery2.brgh, myquery2.mean_score,myquery2.final_count
FROM(
    SELECT myquery.brgh, COUNT(myquery.res_id) as final_count, AVG(myquery.last_score) as mean_score, insp_date
    FROM(
        SELECT BORO as brgh, CAMIS as res_id, SCORE as last_score, MAX(INSPDATE) as insp_date
        FROM webextract3
        GROUP BY CAMIS
    ) as myquery
    GROUP BY
        myquery.brgh
) as myquery2

ORDER BY
    myquery2.mean_score

 * sqlite:///nyc_inspection.db
Done.


myquery2.brgh,myquery2.mean_score,myquery2.final_count
2,10.799032541776604,2281
1,11.437810945273633,9891
3,11.552789699570816,5849
4,11.800629396519808,5417
5,11.87885462555066,912


Violations by cuisine:
we'll look at the difference between the oldest and newest inspection date, treating anything less than 30 days as 30 days (to account for those that were only inspected once, we'll assume everything was open for at least a month).

Since there are so many restaurants, we'll group them by cuisine and do a weighted average by computing 

    (total violations for a cuisine) / (total restaurant-years for that cuisine)

In [100]:
%%sql
SELECT COUNT(*)
FROM (
SELECT *
FROM webextract2
WHERE NOT CUISINECODE IS NULL AND NOT VIOLCODE IS NULL
)


 * sqlite:///nyc_inspection.db
Done.


COUNT(*)
517107


In [7]:
%%sql
DROP TABLE IF EXISTS webextract4;
CREATE TABLE webextract4 AS
    SELECT
        web.*
    FROM
        webextract2 as web,
        (SELECT
            CAMIS
         FROM
             webextract2
         WHERE NOT VIOLCODE IS NULL
         GROUP BY CAMIS) as sub_viol
    WHERE
        web.CAMIS == sub_viol.CAMIS

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [8]:
%%sql
DROP TABLE IF EXISTS webextract5;
CREATE TABLE webextract5 AS
    SELECT
        web.*
    FROM
        webextract2 as web,
        (SELECT *
         FROM(
        (SELECT
            CUISINECODE, COUNT(VIOLCODE) AS count_viol
         FROM
             webextract4
         GROUP BY CUISINECODE)
         )         
         WHERE count_viol>100
        ) as sub_viol
    WHERE
        web.CUISINECODE == sub_viol.CUISINECODE

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [305]:
%%sql
SELECT COUNT(*) FROM webextract5 

 * sqlite:///nyc_inspection.db
Done.


COUNT(*)
522410


In [352]:
%%sql
DROP TABLE IF EXISTS merge_q5;
CREATE TABLE merge_q5 AS
    SELECT vl_table.CUISINECODE AS cs_code ,SUM(vl_table.vl_count)/SUM(vl_months) as viol_per_c
    FROM (
    SELECT
        CUISINECODE, COUNT(VIOLCODE) AS vl_count,((julianday(MAX(INSPDATE))-julianday(MIN(INSPDATE)))/30.0)/12.0 AS vl_months
    FROM
        webextract5
    GROUP BY CAMIS
    ) AS vl_table
    GROUP BY CUISINECODE
    ORDER BY viol_per_c

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [353]:
%%sql
SELECT
    cs.CODEDESC,
    q5.viol_per_c
FROM
    merge_q5 as q5,
    cuisine as cs
WHERE
    cs.CUISINECODE == q5.cs_code

 * sqlite:///nyc_inspection.db
Done.


CODEDESC,viol_per_c
Hotdogs/Pretzels,3.719061625654638
Soups & Sandwiches,5.813184016289135
Hotdogs,5.90746790245385
Donuts,6.006799789302305
"Ice Cream, Gelato, Yogurt, Ices",6.151182163649641
Sandwiches,6.474978099599688
Café/Coffee/Tea,6.630295287296594
"Bottled beverages, including water, sodas, juices, etc.",6.948428440646936
"Juice, Smoothies, Fruit Salads",7.010269585884692
Not Listed/Not Applicable,7.0784951230840685


In [289]:
len(c)


75

Specific violations by cuisine (disproportionate number of violations):
the 20 highest ratios:

    ((cuisine, violation), ratio, count)

In [355]:
%%sql
SELECT * FROM violations LIMIT 5000

 * sqlite:///nyc_inspection.db
Done.


index,STARTDATE,ENDDATE,CRITICALFLAG,VIOLATIONCODE,VIOLATIONDESC
0,1901-01-01 00:00:00,2003-03-23 00:00:00,Y,01A,"Current valid <a onmouseover=""ShowContent('P2','01A'); return true;"" href=""javascript:ShowContent('P2','01A')"">permit</A> , registration or other authorization to operate establishment not available."
1,2003-03-24 00:00:00,2005-02-17 00:00:00,Y,01A,"Current valid <a onmouseover=""ShowContent('P2','01A'); return true;"" href=""javascript:ShowContent('P2','01A')"">permit</A> , registration or other authorization to operate establishment not available."
2,2005-02-18 00:00:00,2007-06-30 00:00:00,Y,01A,"Current valid <a onmouseover=""ShowContent('P2','01A'); return true;"" href=""javascript:ShowContent('P2','01A')"">permit</A> , registration or other authorization to operate establishment not available."
3,2007-07-01 00:00:00,2008-06-30 00:00:00,Y,01A,"Current valid permit, registration or other authorization to operate establishment not available. Violations points are not assessed for Smoke Free Air Act, trans fat, calorie posting or permit and poster violations."
4,2008-07-01 00:00:00,2009-08-01 00:00:00,Y,01A,"Current valid permit, registration or other authorization to operate establishment not available. Violations points are not assessed for Smoke Free Air Act, trans fat, calorie posting or permit and poster violations."
5,1901-01-01 00:00:00,2003-03-23 00:00:00,Y,01B,"Current valid permit, registration or other authorization to operate Temporary Food Service Establishment is not available."
6,2003-03-24 00:00:00,2005-02-17 00:00:00,Y,01B,"Document issued by the Board, Commissioner or Department unlawfully reproduced or altered."
7,2005-02-18 00:00:00,2007-06-30 00:00:00,Y,01B,"Document issued by the Board, Commissioner or Department unlawfully reproduced or altered."
8,2007-07-01 00:00:00,2008-06-30 00:00:00,Y,01B,"Document issued by the Board, Commissioner or Department unlawfully reproduced or altered."
9,2008-07-01 00:00:00,2009-08-01 00:00:00,Y,01B,"Document issued by the Board, Commissioner or Department unlawfully reproduced or altered."


In [31]:
%%sql
DROP TABLE IF EXISTS merge_q6_a;
CREATE TABLE merge_q6_a AS

    SELECT
        table1.CUISINECODE as C_code, table1.VIOLCODE V_code,table1.mycount/1.0/table2.total_per_c/1.0 as internal_ratio, table1.mycount as count_per_cuisine
    FROM
        (SELECT *
         FROM (
             SELECT CUISINECODE, VIOLCODE, COUNT(VIOLCODE) as mycount
             FROM webextract5
             GROUP BY CUISINECODE, VIOLCODE
         ) as pair
        WHERE NOT pair.VIOLCODE IS NULL) as table1,
        (SELECT pair2.CUISINECODE,SUM(mycount) as total_per_c
         FROM(
             SELECT *
             FROM (
                 SELECT CUISINECODE, VIOLCODE, COUNT(VIOLCODE) as mycount
                 FROM webextract5
                 GROUP BY CUISINECODE, VIOLCODE
             ) as pair
             WHERE NOT pair.VIOLCODE IS NULL) pair2
         GROUP BY pair2.CUISINECODE) as table2
    WHERE
        table1.CUISINECODE==table2.CUISINECODE


 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [35]:
%%sql
DROP TABLE IF EXISTS merge_q6_b;
CREATE TABLE merge_q6_b AS

    SELECT
        table1.VIOLCODE as V_code, table1.mycount as total_count,table1.mycount/1.0/table2.mysum/1.0 as external_ratio
    FROM
        (

            SELECT *
         FROM (
             SELECT VIOLCODE, COUNT(VIOLCODE) as mycount
             FROM webextract5
             GROUP BY VIOLCODE
         ) as pair
        WHERE NOT pair.VIOLCODE IS NULL

        ) as table1,
        (
            SELECT SUM(mycount) as mysum
         FROM (
             SELECT VIOLCODE, COUNT(VIOLCODE) as mycount
             FROM webextract5
             GROUP BY VIOLCODE
         ) as pair
        WHERE NOT pair.VIOLCODE IS NULL


        ) as table2

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [61]:
%%sql
DROP TABLE IF EXISTS merge_q6;
CREATE TABLE merge_q6 AS
    SELECT
        q6a.C_code,
        q6a.V_code,
        q6a.internal_ratio/q6b.external_ratio as ratio, 
        q6a.count_per_cuisine
    FROM
        merge_q6_a as q6a,
        merge_q6_b as q6b
    WHERE
        q6a.V_code==q6b.V_code AND
        q6a.count_per_cuisine >100
    ORDER BY ratio DESC
    LIMIT 20

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [62]:
%%sql
SELECT * FROM merge_q6 LIMIT 5

 * sqlite:///nyc_inspection.db
Done.


C_code,V_code,ratio,count_per_cuisine
49,04C,3.2309470543594165,541
14,20D,3.13663970427678,173
51,04A,3.081113896544719,145
29,10E,3.029683576047109,130
43,04A,2.9542073550886085,193


In [19]:
%%sql
SELECT SUM(mycount) as mysum
     FROM (
         SELECT VIOLCODE, COUNT(VIOLCODE) as mycount
         FROM webextract5
         GROUP BY VIOLCODE
     ) as pair
    WHERE NOT pair.VIOLCODE IS NULL
LIMIT 5

 * sqlite:///nyc_inspection.db
Done.


mysum
516713


In [64]:
%%sql
DROP TABLE IF EXISTS merge_q6_c;
CREATE TABLE merge_q6_c AS
    SELECT
        cs.CODEDESC,
        q6.V_code,
        q6.ratio,
        q6.count_per_cuisine
    FROM
        merge_q6 as q6,
        cuisine as cs
    WHERE
        cs.CUISINECODE == q6.C_code

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [67]:
%%sql
SELECT * FROM violations LIMIT 5

 * sqlite:///nyc_inspection.db
Done.


index,STARTDATE,ENDDATE,CRITICALFLAG,VIOLATIONCODE,VIOLATIONDESC
0,1901-01-01 00:00:00,2003-03-23 00:00:00,Y,01A,"Current valid <a onmouseover=""ShowContent('P2','01A'); return true;"" href=""javascript:ShowContent('P2','01A')"">permit</A> , registration or other authorization to operate establishment not available."
1,2003-03-24 00:00:00,2005-02-17 00:00:00,Y,01A,"Current valid <a onmouseover=""ShowContent('P2','01A'); return true;"" href=""javascript:ShowContent('P2','01A')"">permit</A> , registration or other authorization to operate establishment not available."
2,2005-02-18 00:00:00,2007-06-30 00:00:00,Y,01A,"Current valid <a onmouseover=""ShowContent('P2','01A'); return true;"" href=""javascript:ShowContent('P2','01A')"">permit</A> , registration or other authorization to operate establishment not available."
3,2007-07-01 00:00:00,2008-06-30 00:00:00,Y,01A,"Current valid permit, registration or other authorization to operate establishment not available. Violations points are not assessed for Smoke Free Air Act, trans fat, calorie posting or permit and poster violations."
4,2008-07-01 00:00:00,2009-08-01 00:00:00,Y,01A,"Current valid permit, registration or other authorization to operate establishment not available. Violations points are not assessed for Smoke Free Air Act, trans fat, calorie posting or permit and poster violations."


In [73]:
%%sql
DROP TABLE IF EXISTS merge_q6_v;
CREATE TABLE merge_q6_v AS
    SELECT
        q6.CODEDESC,
        vl.VIOLATIONDESC,
        q6.ratio,
        q6.count_per_cuisine
    FROM
        merge_q6_c as q6,
        violations as vl
    WHERE
        vl.VIOLATIONCODE == q6.V_code AND
        julianday(vl.ENDDATE) > julianday('2014-01-01')
    ORDER BY ratio DESC

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [75]:
%%sql
SELECT * FROM merge_q6_v

 * sqlite:///nyc_inspection.db
Done.


CODEDESC,VIOLATIONDESC,ratio,count_per_cuisine
Japanese,Food worker does not use proper utensil to eliminate bare hand contact with food that will not receive adequate additional heat treatment.,3.2309470543594165,541
Café/Coffee/Tea,"“Choking first aid” poster not posted. “Alcohol and pregnancy” warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted. Inspection report sign not posted.",3.13663970427678,173
"Juice, Smoothies, Fruit Salads",Food Protection Certificate not held by supervisor of food operations.,3.081113896544719,145
Donuts,Accurate thermometer not provided in refrigerated or hot holding equipment.,3.029683576047109,130
"Ice Cream, Gelato, Yogurt, Ices",Food Protection Certificate not held by supervisor of food operations.,2.9542073550886085,193
Thai,Thawing procedures improper.,2.624887332286926,151
Irish,"Raw, cooked or prepared food is adulterated, contaminated, cross-contaminated, or not discarded in accordance with HACCP plan.",2.376584992405568,321
Mexican,"Food not cooled by an approved method whereby the internal product temperature is reduced from 140º F to 70º F or less within 2 hours, and from 70º F to 41º F or less within 4 additional hours.",2.3305117992050106,260
Indian,"Food not cooled by an approved method whereby the internal product temperature is reduced from 140º F to 70º F or less within 2 hours, and from 70º F to 41º F or less within 4 additional hours.",2.2529197577139786,112
Chinese,Thawing procedures improper.,2.1918127881756053,1121


In [403]:
%%sql
SELECT
    table1.CUISINECODE, table1.VIOLCODE,table1.mycount/1.0/table2.total_per_c/1.0 as ratio
FROM
    (SELECT *
     FROM (
         SELECT CUISINECODE, VIOLCODE, COUNT(VIOLCODE) as mycount
         FROM webextract5
         GROUP BY CUISINECODE, VIOLCODE
     ) as pair
    WHERE NOT pair.VIOLCODE IS NULL) as table1,
    (SELECT *
     FROM(
         SELECT *
         FROM (
             SELECT VIOLCODE, COUNT(VIOLCODE) as mycount
             FROM webextract5
             GROUP BY VIOLCODE
         ) as pair
         WHERE NOT pair.VIOLCODE IS NULL) pair2) as table2
WHERE
    table1.CUISINECODE==table2.CUISINECODE
ORDER BY ratio DESC   
LIMIT 5

 * sqlite:///nyc_inspection.db
Done.


VIOLCODE,COUNT(VIOLCODE)
None,0
02A,498
02B,24508
02C,550
02D,52


In [379]:
%%sql
(SELECT pair2.CUISINECODE,SUM(mycount)
 FROM(
     SELECT *
     FROM (
         SELECT CUISINECODE, VIOLCODE, COUNT(VIOLCODE) as mycount
         FROM webextract
         GROUP BY CUISINECODE, VIOLCODE
     ) as pair
     WHERE NOT pair.VIOLCODE IS NULL) pair2
 GROUP BY pair2.CUISINECODE) as table2
LIMIT 5

 * sqlite:///nyc_inspection.db
Done.


pair2.CUISINECODE,SUM(mycount)
0,131
1,300
2,1843
3,125458
4,632


In [ ]:
%%sql
DROP TABLE IF EXISTS merge_q5;
CREATE TABLE merge_q5 AS
    SELECT vl_table.CUISINECODE AS cs_code ,SUM(vl_table.vl_count)/SUM(vl_months) as viol_per_c
    FROM (
    SELECT
        CUISINECODE, COUNT(VIOLCODE) AS vl_count,((julianday(MAX(INSPDATE))-julianday(MIN(INSPDATE)))/30.0)/12.0 AS vl_months
    FROM
        webextract5
    GROUP BY CAMIS
    ) AS vl_table
    GROUP BY CUISINECODE
    ORDER BY viol_per_c

In [358]:
%%sql

SELECT * FROM webextract5 WHERE SCORE IS NULL LIMIT 5

 * sqlite:///nyc_inspection.db
Done.


index,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
56200,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234.0,7184443838,27,2013-08-03 00:00:00,F,16A,None,None,None,2014-09-04 06:01:28.403000000
56227,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE,10462.0,7188924968,8,2012-12-31 00:00:00,F,16B,None,None,None,2014-09-04 06:01:28.403000000
56240,40356151,BRUNOS ON THE BOULEVARD,4,88-25,ASTORIA BOULEVARD,11369.0,7183350505,3,2012-02-10 00:00:00,E,16B,None,None,None,2014-09-04 06:01:28.403000000
56268,40356151,BRUNOS ON THE BOULEVARD,4,88-25,ASTORIA BOULEVARD,11369.0,7183350505,3,2014-04-11 00:00:00,F,16B,None,None,None,2014-09-04 06:01:28.403000000
56270,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234.0,7184443838,27,2012-03-09 00:00:00,S,20D,None,None,None,2014-09-04 06:01:28.403000000
